High Availability Infrastructure

**Main.tf**

In [ ]:
# Provider for Cloud Provider
provider "aws" {
    profile = "arijit_aws"
    region = "us-east-1"
}

# Custom VPC for the Instances
resource "aws_vpc" "private_vpc" {
    cidr_block = 10.0.0.0/16
}

# Here we have two Subnets - EC2 subnet (front-end) and DB subnet (back-end)

resource "aws_subnet" "public_subnet" {
  vpc_id     = "${aws_vpc.custom_vpc.id}"       # aws_vpc.<vpc_name>.id
  cidr_block = "10.0.1.0/24"

  tags = {
    Name = "public_subnet"
  }
}

resource "aws_subnet" "public_subnet2" {
  vpc_id     = "${aws_vpc.custom_vpc.id}"       # aws_vpc.<vpc_name>.id
  cidr_block = "10.0.2.0/24"

  tags = {
    Name = "public_subnet2"
  }
}

# Route Table

resource "aws_route_table" "route_table" {
  vpc_id = "${aws_vpc.custom_vpc.id}"

  route {
    cidr_block = "0.0.0.0/0"
    gateway_id = "${aws_internet_gateway.main_ig.id}"
  }

  tags = {
    Name = "route_table"
  }
}

resource "aws_route_table_association" "RT_associate_1" {
  subnet_id      = "${aws_subnet.public_subnet.id}"
  route_table_id = "${aws_route_table.route_table.id}"
}

resource "aws_route_table_association" "RT_associate_2" {
  subnet_id      = "${aws_subnet.public_subnet2.id}"
  route_table_id = "${aws_route_table.route_table.id}"
}

# Internet Gateway

resource "aws_internet_gateway" "main_ig" {
  vpc_id = "${aws_vpc.custom_vpc.id}"

  tags = {
    Name = "main_ig"
  }
}

# Network Access Control List for the two subnets

resource "aws_network_acl" "main_nacl" {
  vpc_id = "${aws_vpc.custom_vpc.id}"
  subnet_ids = ["${aws_subnet.public_subnet.id}", "${aws_subnet.public_subnet2.id}"

  egress {
    protocol   = "ssh"
    rule_no    = 300
    action     = "allow"
    cidr_block = "0.0.0.0/0"
    from_port  = "22"
    to_port    = "22"
  }

  egress {
    protocol   = "tcp"
    rule_no    = 100
    action     = "allow"
    cidr_block = "0.0.0.0/0"
    from_port  = "80"
    to_port    = "80"
  }
  
  egress {
    protocol   = "tcp"
    rule_no    = 200
    action     = "allow"
    cidr_block = "0.0.0.0/0"
    from_port  = "1024"
    to_port    = "65535"
  }

  ingress {
    protocol   = "tcp"
    rule_no    = 100
    action     = "allow"
    cidr_block = "0.0.0.0/0"
    from_port  = "1024"
    to_port    = "65535"
  }

  ingress {
    protocol   = "ssh"
    rule_no    = 300
    action     = "allow"
    cidr_block = "0.0.0.0/0"
    from_port  = "22"
    to_port    = "22"
  }

  ingress {
    protocol   = "tcp"
    rule_no    = 100
    action     = "allow"
    cidr_block = "0.0.0.0/0"
    from_port  = "80"
    to_port    = "80"
  }

  tags = {
    Name = "main_nacl"
  }
}

# The Security group for the EC2 Instances

resource "aws_security_group" "custom_sg" {
  name        = "custom_sg"
  description = "Security group for my web server"
  vpc_id      = "${aws_vpc.private_vpc.id}"

  ingress {
    description = "TLS from VPC"
    from_port   = "80"
    to_port     = "80"
    protocol    = "tcp"
    security_group = ["${aws_security_group.loadbalancer_sg.id}"]
  }

  ingress {
    from_port   = "22"
    to_port     = "22"
    protocol    = "ssh"
    cidr_blocks = "0.0.0.0/0"
  }

  egress {
      from_port = "0"
      to_port = "0"
      protocol = "-1"
      cidr_blocks = ["0.0.0.0/0"]
  }

  tags = {
    Name = "custom_sg"
  }
}

# Security Group for the Load Balancer

resource "aws_security_group" "loadbalancer_sg" {
    name = "loadbalancer_sg"
    description = "SG for load balancer"
    vpc_id = "${aws_vpc.custom_vpc.id}"

    ingress {
        from_port = "80"
        to_port = "80"
        protocol = "tcp"
        cidr_blocks = ["${aws_security_group.loadbalancer_sg.id}"] 
    }

    ingress {
        from_port = "22"
        to_port = "22"
        protocol = "tcp"
        cidr_blocks = ["0.0.0.0/0"]
    }

    egress {
        from_port = "0"
        to_port = "0"
        protocol = "-1"
        cidr_blocks = ["0.0.0.0/0"]

    }
}

# Security Group for the Database MySQL

resource "aws_security_group" "db_sg" {
    name = "db_sg"
    description = "Security group for MySQL RDS"
    vpc_id = "${aws_vpc.custom_vpc.id}"

    ingress = {
        from_port = "3306"
        to_port = "3306"
        protocol ='tcp'
        cidr_blocks = ["0.0.0.0/0"]
    }
}

# EC2 Instance : webserver_1 and webserver_2

resource "aws_instance" "webserver_1" {
    ami = "amazon_linux_ami_id"
    instance_type = "t2.micro" 
    subnet_id = "${aws_subnet.public_subnet.id}"
    vpc_security_group_ids = "${aws_security_group.custom_sg.id}"

    key_name = "mm.pem"
}

resource "aws_instance" "webserver_2" {
    ami = "amazon_linux_ami_id"
    instance_type = "t2.micro" 
    subnet_id = "${aws_subnet.public_subnet2.id}"
    vpc_security_group_ids = "${aws_security_group.custom_sg.id}"

    key_name = "mm.pem"
}

# Load Balancer for the Incoming traffic

resource "aws_lb" "webserver_lb" {
  name               = "webserver_lb"
  internal           = false
  load_balancer_type = "application"
  security_groups    = ["${aws_security_group.loadbalancer_sg.id}"]
  subnets            = ["${aws_subnet.public_subnet.id}", "${aws_subnet.public_subnet2.id}"]
}

# Load Balancer listener for connection requests

resource "aws_lb_listener" "front_end" {
  load_balancer_arn = "${aws_instance.webserver_1.id}"
  port              = "80"
  protocol          = "HTTP"
  # ssl_policy        = "ELBSecurityPolicy-2016-08"
  # certificate_arn   = "arn:aws:iam::187416307283:server-certificate/test_cert_rab3wuqwgja25ct3n4jdj2tzu4"

  default_action {
    type             = "forward"
    target_group_arn = "${aws_lb_target_group.webserver_tg.id}"
  }
}

# Load Balancer Target Group to route requests to one or more registered targets

resource "aws_lb_target_group" "webserver_tg" {
     name     = "webserver_tg"
     port     = "80"
     protocol = "HHTP"
     vpc_id   = "${aws_vpc.private_vpc.id}"
}

# Load Balancer Traget Group Attachment

resource "aws_lb_target_group_attachment" "TG_attach_1" {
  target_group_arn = "${aws_lb_target_group.webserver_tg.arn}"
  target_id        = "${aws_instance.webserver_1.id}"
  port             = "80"
}

resource "aws_lb_target_group_attachment" "TG_attach_2" {
  target_group_arn = "${aws_lb_target_group.webserver_tg.arn}"
  target_id        = "${aws_instance.webserver_2.id}"
  port             = "80"
}

# Subnet Group for MySQL

resource "aws_db_subnet_group" "mysql_subnet_group" {
  name       = "mysql_subnet_group"
  subnet_ids = ["${aws_subnet.public_subnet.id}", "${aws_subnet.public_sunet2.id}"]

  tags = {
    Name = "mysql_subnet_group"
  }
}

# Database Instance to host MySQL application

resource "aws_db_instance" "webserver_mysql" {
  allocated_storage    = "20"
  storage_type         = "gp2"    # general purpose ssd
  engine               = "mysql"
  engine_version       = "5.7.22"
  instance_class       = "db.t2.micro"

  name                 = "webserver_mysql"
  username             = "user1"
  password             = "mysqluser1"
  parameter_group_name = "default.mysql5.7"

  skip_final_snapshot = "true"
  db_subnet_group_name = "${aws_db_subnet_group.mysql_subnet_group.id}"

  vpc_security_group_ids = ["${aws_security_group.db_sg.id}"]
}
